In [ ]:
#!python -m pip install -q git+https://github.com/wilhelm-lab/dlomix

In [ ]:
#!python -m pip install wandb

# Preprocess the data

In [ ]:
import pandas as pd
import re
import numpy as np
import csv

## Read the input data

In [ ]:
input_file1 = "ptm/input_data/TUM_mod_citrullination_l.parquet"
input_file2 = "ptm/input_data/TUM_mod_citrullination_2.parquet"

## Create dataframe

In [ ]:
tmp_data1 = pd.read_parquet(input_file1, engine='pyarrow')
tmp_data2 = pd.read_parquet(input_file2, engine='pyarrow')
data = pd.concat([tmp_data1,tmp_data2])

In [ ]:
data.head(10)

## Divide data into HCD and CID

In [ ]:
data_HCD = data[data['fragmentation']=='HCD']
data_CID = data[data['fragmentation']=='CID']

In [ ]:
data_HCD.head(10)

## Sort for modified_sequence and sort andromeda_score from highest to lowest

In [ ]:
data_HCD = data_HCD.sort_values(['modified_sequence', 'andromeda_score'], ascending=[True, False])

In [ ]:
data_HCD.head(10)

## Select only the top 5 modified_sequences with highest score

In [ ]:
data_HCD_5 = data_HCD.sort_values(['modified_sequence', 'andromeda_score'], ascending=[True, False]).groupby('modified_sequence').head(5)

In [ ]:
data_HCD_5.reset_index(inplace=True)

In [ ]:
data_HCD_5.head(10)

In [ ]:
data_CID_5 = data_CID.sort_values(['modified_sequence', 'andromeda_score'], ascending=[True, False]).groupby('modified_sequence').head(5)

In [ ]:
data_CID_5.reset_index(inplace=True)

In [ ]:
data_CID_5.head(10)

## Calculate 10% of the dataframes to create a test set

In [ ]:
ten_percent_HCD = int(len(data_HCD_5)*0.1)
ten_percent_HCD

In [ ]:
ten_percent_CID = int(len(data_CID_5)*0.1)
ten_percent_CID

## Create TEST set

In [ ]:
test_HCD = data_HCD_5.head(ten_percent_HCD)

In [ ]:
index_test_HCD = test_HCD.index
index_test_HCD

In [ ]:
test_CID = data_CID_5.head(ten_percent_CID)

In [ ]:
index_test_CID = test_CID.index
index_test_CID

## Create TRAIN_VAL set

In [ ]:
train_val_HCD = data_HCD_5.drop(data_HCD_5.index[index_test_HCD])
train_val_HCD

In [ ]:
train_val_CID = data_CID_5.drop(data_CID_5.index[index_test_CID])
train_val_CID

## Check if TEST and TRAIN_VAL have intersecting values

## ##TODO: the maximal overlapp is 5. Put the overlapped sequence into the set where the rest of the same sequences are.

In [ ]:
pd.Series(list(set(train_val_HCD['modified_sequence']).intersection(set(test_HCD['modified_sequence']))))

## Change modification e.g R[UNIMOD:7] to r

In [ ]:
def changeMod(input_data):
        for index, row in input_data.iterrows():
            sequence = row['modified_sequence']
            open_bracket = find(sequence,'[')
            tmp_seq =""
            new_sequence = ""
            if len(open_bracket)>=1:
                for index_mod in open_bracket:
                    modified_AA = sequence[index_mod-1]
                    if modified_AA == 'M':
                        tmp_seq = sequence[:index_mod-1] + 'M(ox)' + sequence[index_mod:]
                        new_sequence = re.sub("[\[].*?[\]]", "", tmp_seq)
        
                    else:
                        tmp_seq = sequence[:index_mod-1] + modified_AA.lower() + sequence[index_mod:]
                        new_sequence = re.sub("[\(\[].*?[\)\]]", "", tmp_seq)
                input_data.at[index, 'modified_sequence'] = new_sequence
        return input_data
    
def find(s, ch):
        return [i for i, ltr in enumerate(s) if ltr == ch]

In [ ]:
test_HCD = changeMod(test_HCD)
train_val_HCD = changeMod(train_val_HCD)

In [ ]:
test_CID = changeMod(test_CID)
train_val_CID = changeMod(train_val_CID)

## Remove specific modifications e.g Q, K

In [ ]:
def removeChar(data,removeChar:str):
    tmp_remove = removeChar+"\\["
    filter = data['modified_sequence'].str.contains(tmp_remove)
    filtered_df = data[~filter]
    return filtered_df.reset_index(drop=True)

In [ ]:
test_HCD = removeChar(test_HCD,'K')
test_HCD = removeChar(test_HCD,'Q')

train_val_HCD = removeChar(train_val_HCD,'K')
train_val_HCD = removeChar(train_val_HCD,'Q')

In [ ]:
test_CID = removeChar(test_CID,'K')
test_CID = removeChar(test_CID,'Q')

train_val_CID = removeChar(train_val_CID,'K')
train_val_CID = removeChar(train_val_CID,'Q')

## Rename columns

In [ ]:
train_val_HCD.rename(columns={'intensities_raw':'intensities','modified_sequence': 'sequence', 'collision_energy_aligned_normed': 'collision_energy', 'precursor_charge_onehot':'precursor_charge_onehot'}, inplace=True)
test_HCD.rename(columns={'intensities_raw':'intensities','modified_sequence': 'sequence', 'collision_energy_aligned_normed': 'collision_energy', 'precursor_charge_onehot':'precursor_charge_onehot'}, inplace=True)

In [ ]:
train_val_CID.rename(columns={'intensities_raw':'intensities','modified_sequence': 'sequence', 'collision_energy_aligned_normed': 'collision_energy', 'precursor_charge_onehot':'precursor_charge_onehot'}, inplace=True)
test_CID.rename(columns={'intensities_raw':'intensities','modified_sequence': 'sequence', 'collision_energy_aligned_normed': 'collision_energy', 'precursor_charge_onehot':'precursor_charge_onehot'}, inplace=True)

## Change format of precurser_charge from [1 0 0 1 0] to [1, 0, 0, 1, 0]

In [ ]:
train_val_HCD['intensities'] = train_val_HCD['intensities'].apply(lambda a: np.array2string(a, separator=', '))
train_val_HCD['precursor_charge_onehot'] = train_val_HCD['precursor_charge_onehot'].apply(lambda a: np.array2string(a, separator=', '))

test_HCD['intensities'] = test_HCD['intensities'].apply(lambda a: np.array2string(a, separator=', '))
test_HCD['precursor_charge_onehot'] = test_HCD['precursor_charge_onehot'].apply(lambda a: np.array2string(a, separator=', '))

In [ ]:
train_val_CID['intensities'] = train_val_CID['intensities'].apply(lambda a: np.array2string(a, separator=', '))
train_val_CID['precursor_charge_onehot'] = train_val_CID['precursor_charge_onehot'].apply(lambda a: np.array2string(a, separator=', '))

test_CID['intensities'] = test_CID['intensities'].apply(lambda a: np.array2string(a, separator=', '))
test_CID['precursor_charge_onehot'] = test_CID['precursor_charge_onehot'].apply(lambda a: np.array2string(a, separator=', '))

## Write TEST and TRAIN_VAL to .csv file

In [ ]:
train_val_HCD.to_csv('ptm/output/train_val_hcd.csv', encoding='utf-8', index=False)
test_HCD.to_csv('ptm/output/test_hcd.csv', encoding='utf-8', index=False)

In [ ]:
train_val_CID.to_csv('ptm/output/train_val_cid.csv', encoding='utf-8', index=False)
test_CID.to_csv('ptm/output/test_cid.csv', encoding='utf-8', index=False)

# Intensity Prediction

In [1]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
print([x for x in dir(dlomix) if not x.startswith("_")])

wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


['META_DATA', 'constants', 'data', 'eval', 'layers', 'losses', 'models', 'pipelines', 'reports', 'utils']


## 0. Import and Initialize Weights and Biases

In [2]:
import wandb
from wandb.keras import WandbCallback

In [3]:
# enter project name
project_name = 'dlomix_intensity'

## 1. Load Data

### 1.1 Load training and validation data

In [4]:
from dlomix.data import IntensityDataset

In [5]:
#TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/tasks/intensity/example_datasets/Intensity/proteomeTools_train_val.csv'
TRAIN_DATAPATH = 'D:\\Uni\\Masterarbeit\\dlomix\\ptm\\output\\train_val_hcd.csv'

BATCH_SIZE = 64

int_data = IntensityDataset(data_source=TRAIN_DATAPATH, seq_length=30, batch_size=BATCH_SIZE,
                            collision_energy_col='collision_energy', val_ratio=0.2, test=False)

In [6]:
"Training examples", BATCH_SIZE * len(int_data.train_data)

('Training examples', 17408)

In [7]:
"Validation examples", BATCH_SIZE * len(int_data.val_data)

('Validation examples', 4352)

### 1.2 Load weights from HDF5 file for embedding layer

In [ ]:
import h5py

In [ ]:
f = h5py.File('ptm/weights/weights_163_0.11385.hdf5', 'r+')

In [ ]:
group = f['model_weights']
data_file = group['embedding']
for group in data_file.keys() :
    for dset in data_file[group].keys():      
        weights_hpf5 = data_file[group][dset][:]
weights_hpf5.shape

In [ ]:
f.close()

In [ ]:
newrow = np.random.uniform(-1, 1, size=(1, 32))
weights_hpf5 = np.vstack([weights_hpf5, newrow])

## 2. Models

In [8]:
from dlomix.models import PrositIntensityPredictor
import keras
import tensorflow as tf
import numpy as np

### 2.1 Random initialization of the embedding layer

#### 2.1.1 Create random uniform weight matrix

In [ ]:
rand_uni_weights = np.random.uniform(-1, 1, size=(24, 32))
rand_uni_weights.shape

#### 2.1.2 Initialize random model on input example to modify weights before training

In [9]:
sequences = [[i] for i in int_data.sequences]

In [10]:
model_random = PrositIntensityPredictor(seq_length=30)
model_random(inputs={'sequence':sequences, 'collision_energy':tf.convert_to_tensor(int_data.collision_energy, dtype=np.float32), 'precursor_charge':tf.convert_to_tensor(int_data.precursor_charge, dtype=np.float32)})

<tf.Tensor: shape=(21727, 174), dtype=float32, numpy=
array([[ 6.8178315e-07,  1.0845733e-06, -2.7902598e-07, ...,
        -2.1307061e-07,  2.9306048e-06, -7.9584578e-07],
       [ 3.1844309e-07, -4.6067004e-07,  5.8303090e-07, ...,
        -1.6957323e-06,  2.1660853e-06,  2.9405576e-06],
       [ 3.2017454e-07, -4.5950918e-07,  5.8120082e-07, ...,
        -1.6939072e-06,  2.1637863e-06,  2.9417624e-06],
       ...,
       [-5.1038683e-07, -4.1841339e-08,  1.4766320e-06, ...,
        -1.7317437e-06,  7.5229209e-06,  1.8147668e-06],
       [-5.1038683e-07, -4.1841339e-08,  1.4766320e-06, ...,
        -1.7317437e-06,  7.5229209e-06,  1.8147668e-06],
       [-5.1038683e-07, -4.1841339e-08,  1.4766320e-06, ...,
        -1.7317437e-06,  7.5229209e-06,  1.8147668e-06]], dtype=float32)>

In [ ]:
#model_random = PrositIntensityPredictor(seq_length=30)
#model_random(inputs={'sequence':[['EYYLRYLEK']], 'collision_energy':tf.convert_to_tensor([[35.0]], dtype=np.float32), 'precursor_charge':tf.convert_to_tensor([[0, 1, 0, 0, 0, 0]], dtype=np.float32)})

#### 2.1.3 Assign random weight matrix to model

In [ ]:
model_random.layers[1].set_weights([rand_uni_weights])    

In [12]:
model_random.layers[1].get_weights()[0]

(24, 32)

### 2.2 Copy row of M to M(ox) and R to r

#### 2.2.1 Create random uniform weight matrix

In [ ]:
rand_uni_weights = np.random.uniform(-1, 1, size=(22, 32))

#### 2.2.2 Copy vector of M to M(ox) and R to r

In [ ]:
row_M = rand_uni_weights[12,]
row_R = rand_uni_weights[16,]

In [ ]:
row_Mox = row_M
row_r = row_R
rand_uni_weights = np.vstack([rand_uni_weights, row_Mox])
rand_uni_weights = np.vstack([rand_uni_weights, row_r])

In [ ]:
rand_uni_weights.shape

#### 2.2.3 Initialize copy model on input example to modify weights before training

In [ ]:
model_copy = PrositIntensityPredictor(seq_length=30)
model_copy(inputs={'sequence':[['EYYLRYLEK']], 'collision_energy':tf.convert_to_tensor([[35.0]], dtype=np.float32), 'precursor_charge':tf.convert_to_tensor([[0, 1, 0, 0, 0, 0]], dtype=np.float32)})

#### 2.2.4 Assign weight matrix to model

In [ ]:
model_copy.layers[1].set_weights([rand_uni_weights])   

### 2.3 Select row with best weights and assign to M(ox) and r

## 3. Training

### 3.1 Training random model

In [ ]:
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
tf.get_logger().setLevel('ERROR')

In [ ]:
# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom timedelta metric
model_random.compile(optimizer=optimizer,
              loss=masked_spectral_distance,
              metrics=['mse',masked_pearson_correlation_distance])

In [ ]:
wandb.init(project=project_name, name='random model')
history = model_random.fit(int_data.train_data,
                    validation_data=int_data.val_data,
                    epochs=15,callbacks=[WandbCallback(save_model=False)])

### 3.2 Training copy model

In [ ]:
# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom timedelta metric
model_copy.compile(optimizer=optimizer,
              loss=masked_spectral_distance,
              metrics=['mse',masked_pearson_correlation_distance])

In [ ]:
wandb.init(project=project_name, name='copy model')
history = model_copy.fit(int_data.train_data,
                    validation_data=int_data.val_data,
                    epochs=15,callbacks=[WandbCallback(save_model=False)])

In [ ]:
# Mark the run as finished
wandb.finish()

### 3.3 Training clever copy model

## 4. Save the model

In [ ]:
#save_path = "./output/rtmodel.hdf5"
#model.save_weights(save_path)

In [ ]:
#trained_model = RetentionTimePredictor(seq_length=30)
#trained_model.load_weights(save_path)

## 5. Testing and Reporting

In [ ]:
# create the dataset object for test data

#TEST_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/tasks/intensity/example_datasets/Intensity/proteomeTools_test.csv'
TEST_DATAPATH = 'D:\\Uni\\Masterarbeit\\dlomix\\ptm\\output\\test_hcd.csv'

test_int_data = IntensityDataset(data_source=TEST_DATAPATH,
                              seq_length=30, collision_energy_col='collision_energy',batch_size=32, test=True)

In [ ]:
# use model.predict from keras directly on the testdata

predictions = model.predict(test_int_data.test_data)

In [ ]:
from dlomix.reports import IntensityReport

# create a report object by passing the history object and plot different metrics
report = IntensityReport(output_path="./output", history=history)

In [ ]:
# you can generate a complete report for intensity by calling generate_report
# the function takes the test dataset object and the predictions as arguments

report.generate_report(test_int_data, predictions)

In [ ]:
# you can also manually see the results by calling other utility functions
from dlomix.reports.postprocessing import normalize_intensity_predictions


predictions_df = report.generate_intensity_results_df(test_int_data, predictions)
predictions_df.head()

In [ ]:
predictions_acc = normalize_intensity_predictions(predictions_df)
predictions_acc.head()

In [ ]:
predictions_acc['spectral_angle'].describe()

In [ ]:
import seaborn as sns

sns.violinplot(predictions_acc['spectral_angle'])